In [ ]:
import tensorflow_datasets as tfds
import csv
import sys
import tensorflow as tf

wiki_tfds = tfds.load('wikipedia/20190301.en', split='train')

In [ ]:
sample = wiki_tfds.take(2)
for x in sample:
  print(x)

In [ ]:
csv.field_size_limit(sys.maxsize)

joined_csv_file_path = 'home/shivguptashi/wikidata/joined_table.csv'
id_to_info = {}
title_to_info = {}
with gfile.Open(joined_csv_file_path, 'r') as f:
  csvreader = csv.reader(f, delimiter=',')
  it = 0
  for row in csvreader:
    title = row[-7].strip('|')
    #print(title)
    title_to_info[title] = row
    it += 1
    if it % 1000 == 0:
      print(it)


In [ ]:
num_rows_in_info = len(title_to_info['Tatrapan'])
print(num_rows_in_info)
print(num_rows_in_info - 7 - 4)

In [ ]:
from operator import itemgetter

def map_to_features(title, text):
  str_title = title.numpy().decode('utf-8')
  #try:
  if str_title not in title_to_info:
    return (title, text, -1)
  all_entries = [float(y) for y in title_to_info[str_title][4:num_rows_in_info-7]]
  index, _ = max(enumerate(all_entries), key=itemgetter(1))
  return (title, text, index)
  #except:
  #  return (title, text, -1)

wiki_tfds = wiki_tfds.map(lambda x: (x['title'], x['text']), num_parallel_calls=16)
wiki_tfds = wiki_tfds.map(lambda x, y: tf.py_function(map_to_features, [x, y], [tf.string, tf.string, tf.int32]), num_parallel_calls=16)


In [ ]:
save_path = 'home/shivguptashi/wikidata/wiki_tfds_with_topic'
tf.data.Dataset.save(wiki_tfds, save_path)
#from functools import partial
#def filter_f(x, i):
#  try:
#    str_title = x.numpy().decode('utf-8')
#    all_entries = [int(y) for y in title_to_info[str_title][4:num_rows_in_info-7]]
#    print(all_entries)
#    if int(title_to_info[str_title][i]) >= max(all_entries):
#      return True
#    return False
#  except:
#    return False
#
#categories_tfds = []
#for i in range(4, num_rows_in_info - 7):
  #categories_tfds.append(wiki_tfds.filter(lambda x, y: tf.py_function(partial(filter_f, i=i), [x,], [tf.bool,])[0]))

In [ ]:
sample_tfds = wiki_tfds.take(2)
for x in sample_tfds:
  print(x[0])

In [ ]:
sample_category_tfds = categories_tfds[0].take(2)
for x in sample_category_tfds:
  print(x)

In [ ]:

wiki_tfds_with_topic = tf.data.Dataset.load(save_path)
sample_tfds_with_topic = wiki_tfds_with_topic.take(10)
sample_wiki_tfds_with_topic = sample_tfds_with_topic.filter(lambda x, y, z: tf.py_function(filter_f, [z], [tf.bool])[0])
for x in sample_wiki_tfds_with_topic:
  print(x)

In [ ]:
from functools import partial
def filter_f(topic_ind, i):
  if topic_ind == i:
    return True
  return False

topic_wise_save_path = 'home/shivguptashi/wikidata/topic_wise_tfds'
for i in range(58, num_rows_in_info - 7):
  ind = i - 4
  cur_topic_save_path = topic_wise_save_path + '_topic_' + str(ind)
  filtered_wiki_tfds = wiki_tfds_with_topic.filter(lambda x, y, z: tf.py_function(partial(filter_f, i=ind), [z], [tf.bool])[0])
  print('index:', ind)
  tf.data.Dataset.save(filtered_wiki_tfds, cur_topic_save_path)

In [ ]:
print(num_rows_in_info)